In [2]:
def preprocess_text(text):
    """Preprocess text by keeping all characters as literals"""
    # Convert escape sequences to literal characters
    processed = text.encode('unicode_escape').decode('utf-8')
    return processed

In [3]:
def caesar_cipher_encrypt(text, alphabet, shift):
    encrypted_text = ""
    for char in text:
        if char in alphabet:

            new_index = (alphabet.index(char) + shift) % len(alphabet)
            encrypted_text += alphabet[new_index]
        else:
            encrypted_text += char
    return encrypted_text

In [4]:
def caesar_cipher_decrypt(text, alphabet, shift):
    decrypted_text = ""
    for char in text:
        if char in alphabet:
            new_index = (alphabet.index(char) - shift) % len(alphabet)
            decrypted_text += alphabet[new_index]
        else:
            decrypted_text += char
    return decrypted_text

In [6]:
def encode_key(key, alphabet):
    """Convert numeric key to alphabetic representation"""
    key_str = str(key)
    encoded = ""
    for digit in key_str:
        # Map each digit to character (0->first char, 1->second char, etc)
        encoded += alphabet[int(digit)]
    return encoded

def decode_key(encoded_key, alphabet):
    """Convert alphabetic key back to number"""
    decoded = ""
    for char in encoded_key:
        # Map each character back to digit
        decoded += str(alphabet.index(char))
    return int(decoded)

def enhanced_caesar_encrypt(text, alphabet, shift):
    """Encrypt both text and key"""
    # First encrypt the text
    encrypted_text = caesar_cipher_encrypt(text, alphabet, shift)
    # Then encode the key
    encoded_key = encode_key(shift, alphabet)
    # Combine with delimiter
    return f"{encoded_key}##{encrypted_text}"

def enhanced_caesar_decrypt(encrypted_message, alphabet):
    """Decrypt message containing encoded key"""
    # Split key and text
    encoded_key, encrypted_text = encrypted_message.split("##")
    # Decode the key
    shift = decode_key(encoded_key, alphabet)
    # Decrypt the text
    return caesar_cipher_decrypt(encrypted_text, alphabet, shift)

# Example usage:
shift_key = 3
plaintext = "pneumonoultramicroscopicsilicovolcanoconiosis"
processed_plaintext = preprocess_text(plaintext)
alphabet = r"`,.pyfgcrl/=\aoeuidhtns-;qjkxbmwvz"
processed_alphabet = preprocess_text(alphabet)

# Encrypt
encrypted = enhanced_caesar_encrypt(processed_plaintext, processed_alphabet, shift_key)
print(f"Encrypted: {encrypted}")

# Decrypt
decrypted = enhanced_caesar_decrypt(encrypted, processed_alphabet)
print(f"Decrypted: {decrypted}")

Encrypted: p##g;dhzi;ih\-=uzt/=iq/igt/qt\t/i,i\/u;i/i;tiqtq
Decrypted: pneumonoultramicroscopicsilicovolcanoconiosis
